# Reverse Mode Motivation

- Forward mode allows efficient computation of the partial derivatives of multiple functions.
- Reverse mode allows efficient computation of the gradient of a single function.
- In NN: we wish to calculate the slope of each weight w/r/t a loss function.
- Backprop: begin with the output layer and sequentially updates weights to the input layer.
- AD a necessary aspect of contemporary ML because it allows for us to make this calculation.

For a given input $u$, output $w_1$, and a yet-to-be calculated variable $s$:

Forward mode chain rule:

$$\frac{\partial w}{\partial t} = \frac{\partial w}{\partial u_1} * \frac{\partial u_1}{\partial t} + \frac{\partial w}{\partial u_2} * \frac{\partial u_2}{\partial t} + ...  $$

Backward chain rule (inversion of composite functions by flipping numerator and denominator):

$$\frac{\partial s}{\partial u} = \frac{\partial w_1}{\partial u} * \frac{\partial s}{\partial w_1} + \frac{\partial w_2}{\partial u} * \frac{\partial s}{\partial w_2} + ... $$


- Graph theory intuition most useful.

# Slide 2 & 3: Code for NN

In [ ]:
#Summer Yuan

from Dotua.rautodiff import rAutoDiff as rad
from Dotua.roperator import rOperator as op
import random

rad = rad()

class NeuralNetwork():
    def __init__(self, input_vals, input_bias, hidden_bias, num_hidden, output, learning_rate = 0.1):
        self.input_vals = input_vals
        self.input_bias = input_bias
        self.hidden_bias = hidden_bias
        self.num_hidden = num_hidden
        self.output = output
        self.learning_rate = learning_rate

        self.weights_tohidden = [None] * num_hidden
        for i in range(num_hidden):
            self.weights_tohidden[i] = []
            for j in range(len(input_vals)):
                
                #DOTUA USED HERE: to initalize the weights from the input to hidden layer.
                w = rad.create_rscalar(random.random())
                self.weights_tohidden[i].append(w)
                self.weights_tooutput = [None] * len(output)
                
        for i in range(len(output)):
            self.weights_tooutput[i] = []
            
            #DOTUA USED HERE: to initialize weights from hidden to output.
            w = rad.create_rscalar(random.random())
            for j in range(num_hidden):
                self.weights_tooutput[i].append(w)

In [ ]:
    def train(self, input_vals, output_vals):
        self.input_vals = input_vals
        self.output = output_vals

        #calculation of hidden layer.
        self.hidden_layer = []
        for i in range(self.num_hidden):
            h = 0
            for j in range(len(self.weights_tohidden[i])):
                h = h + self.weights_tohidden[i][j] * self.input_vals[j]
            h = h + self.input_bias
            self.hidden_layer.append(1/(1+op.exp(-h)))

        # To calculate the output layer neurons and error
        error = 0
        for i in range(len(self.output)):
            o = 0
            for j in range(len(self.weights_tooutput[i])):
                o = o + self.weights_tooutput[i][j] * self.hidden_layer[j]
            o = o + self.hidden_bias
            o = 1/(1+op.exp(-o))
            error = error + (o - self.output[i]) ** 2

        # To update weights from hidden layer to output layer
        for i in range(len(self.weights_tooutput)):
            for j in range(len(self.weights_tooutput[i])):
                
                #DOTUA USED HERE:
                d = rad.partial(error, self.weights_tooutput[i][j])
                self.weights_tooutput[i][j] = self.weights_tooutput[i][j] - d * self.learning_rate

        # To update weights from input layer to hidden layer
        for i in range(len(self.weights_tohidden)):
            for j in range(len(self.weights_tooutput[i])):
                
                #DOTUA USED HERE:
                d = rad.partial(error, self.weights_tohidden[i][j])
                self.weights_tohidden[i][j] = self.weights_tohidden[i][j] - d * self.learning_rate


# Slide 4: NN Output

![](images/iris_pca.png) | ![](images/cm_nnexample.png)